In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
from stratosphere import Stratosphere
from stratosphere.utils.inspect_flows import InspectFlows
from stratosphere import options

In [3]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100

In [235]:
s = Stratosphere("sqlite:////shared/data/samples/vk-person.db")
df = s.db.pandas("select * from flows")
ifs = InspectFlows(df)

■ Stratosphere v0.0.1 (https://stratosphere.dev) initialized


In [236]:
df = ifs.overview()


Overview: 142 flows (13.21 seconds) distributed as follows:


,host,port,method,flow_request_headers_content_type,flow_response_headers_content_type,count
0,st6-23.vk.com,443,GET,NaN,application/x-javascript,37
1,st6-23.vk.com,443,GET,NaN,text/css,25
2,vk.com,443,GET,NaN,application/x-javascript,22
3,sun6-20.userapi.com,443,GET,NaN,image/jpeg,7
4,api.vk.com,443,POST,application/x-www-form-urlencoded,application/json; charset=utf-8,5
...,...,...,...,...,...,...
19,trk.mail.ru,443,GET,NaN,image/gif,1
20,queuev4.vk.com,443,POST,application/x-www-form-urlencoded,application/json; charset=UTF-8,1
21,vk.com,443,GET,NaN,image/jpeg,1
22,queuev4.vk.com,443,GET,NaN,text/html; charset=windows-1251,1


In [237]:
df = ifs.analyze()
df[df.response_is_json]

,flow_capture_timestamp,host_port_method,url_path_beginswith,response_is_text,response_is_json,flow_response_content_startswith
77,2023-04-08 19:10:17.119075,vk.com:443@POST,/al_audio.php,True,True,"b'{""payload"":[0,[[[456245462,200'"
78,2023-04-08 19:10:17.162656,vk.com:443@POST,/easter-eggs,True,True,"b'{""payload"":[0,[[]]],""statsMeta'"
88,2023-04-08 19:10:18.041233,login.vk.com:443@POST,/,True,True,"b'{""type"":""okay"",""data"":{""access'"
91,2023-04-08 19:10:18.268180,api.vk.com:443@POST,/method/account.getToggles,True,True,"b'{""response"":{""version"":1680866'"
92,2023-04-08 19:10:18.330312,vk.com:443@POST,/al_im.php,True,True,"b'{""payload"":[0,[""empty""]],""stat'"
...,...,...,...,...,...,...
94,2023-04-08 19:10:18.419399,api.vk.com:443@POST,/method/account.getEcosystemSe,True,True,"b'{""response"":[{""id"":1,""title"":""'"
95,2023-04-08 19:10:18.469404,api.vk.com:443@POST,/method/account.getProfileNavi,True,True,"b'{""response"":{""account_navigati'"
96,2023-04-08 19:10:18.545981,vk.com:443@POST,/al_im.php,True,True,"b'{""payload"":[0,[{""friends"":[],""'"
132,2023-04-08 19:10:21.722354,api.vk.com:443@POST,/method/execute,True,True,"b'{""response"":{""messages.getGrou'"


In [238]:
ifs.analyze(patterns=['Kramodanov'])

,flow_capture_timestamp,host_port_method,url_path_beginswith,response_is_text,response_is_json,flow_response_content_startswith,n_matches
0,2023-04-08 19:10:09.179443,vk.com:443@GET,/kramodanov,True,False,"b""<!DOCTYPE html>\n<html lang='e""",1


In [239]:
ifs.df[ifs.df.flow_request_url.str.startswith('https://vk.com') & ifs.df.flow_response_headers_content_type.astype(str).str.startswith('text/html')]

,flow_id,flow_index,flow_request_host,flow_request_port,flow_request_method,flow_request_pretty_url,flow_request_url,flow_request_http_version,flow_request_headers,flow_request_headers_content_type,flow_request_content,flow_request_timestamp_start,flow_response_status_code,flow_response_http_version,flow_response_headers,flow_response_headers_content_type,flow_response_content,flow_response_timestamp_start,flow_capture_timestamp
0,aac65a493e82441eac9dd56d781fd679,0,vk.com,443,GET,https://vk.com/kramodanov,https://vk.com/kramodanov,HTTP/2.0,"{""pragma"": ""no-cache"", ""cache-control"": ""no-cache"", ""sec-ch-ua"": ""\""Brave\"";v=\""111\"", \""Not(A:B...",None,b'',2023-04-08 19:10:08.232682,200,HTTP/2.0,"{""server"": ""kittenx"", ""date"": ""Sat, 08 Apr 2023 19:10:08 GMT"", ""content-type"": ""text/html; chars...",text/html; charset=windows-1251,"b'<!DOCTYPE html>\n<html lang=\'en\' dir=\'ltr\'>\n<head>\n<meta http-equiv=""X-UA-Compatible"" c...",2023-04-08 19:10:08.845221,2023-04-08 19:10:09.179443


In [240]:
def decode(s, encodings=("ascii", "utf-8", "latin-1")):
    for encoding in encodings:
        try:
            return s.decode(encoding)
        except UnicodeDecodeError:
            pass
    return s.decode("ignore")

df = s.db.pandas("select * from flows")
content = decode(df.iloc[0].flow_response_content)

In [241]:
ifs.df[ifs.df.flow_request_url.str.startswith('https://vk.com') & ifs.df.flow_response_headers_content_type.astype(str).str.startswith('text/html')]


,flow_id,flow_index,flow_request_host,flow_request_port,flow_request_method,flow_request_pretty_url,flow_request_url,flow_request_http_version,flow_request_headers,flow_request_headers_content_type,flow_request_content,flow_request_timestamp_start,flow_response_status_code,flow_response_http_version,flow_response_headers,flow_response_headers_content_type,flow_response_content,flow_response_timestamp_start,flow_capture_timestamp
0,aac65a493e82441eac9dd56d781fd679,0,vk.com,443,GET,https://vk.com/kramodanov,https://vk.com/kramodanov,HTTP/2.0,"{""pragma"": ""no-cache"", ""cache-control"": ""no-cache"", ""sec-ch-ua"": ""\""Brave\"";v=\""111\"", \""Not(A:B...",None,b'',2023-04-08 19:10:08.232682,200,HTTP/2.0,"{""server"": ""kittenx"", ""date"": ""Sat, 08 Apr 2023 19:10:08 GMT"", ""content-type"": ""text/html; chars...",text/html; charset=windows-1251,"b'<!DOCTYPE html>\n<html lang=\'en\' dir=\'ltr\'>\n<head>\n<meta http-equiv=""X-UA-Compatible"" c...",2023-04-08 19:10:08.845221,2023-04-08 19:10:09.179443


In [255]:
ifs.df[ifs.df.flow_request_url.str.startswith('https://vk.com') & ifs.df.flow_response_headers_content_type.astype(str).str.startswith('text/html')]

df.iloc[0].flow_response_content
country = re.compile(',"country":{.*?"title":"(.*?)"}').search(content)[1]
city = re.compile(',"city":{.*?"title":"(.*?)"}').search(content)[1]
bdate = re.compile(',"bdate":"(.*?)"').search(content)[1]
skype = re.compile(',"skype":"(.*?)"').search(content)[1]
photo = re.compile("photo_medium_rec.*?src=(.*?),").search(content)[1] #.replace('\\','')

photo.replace('\\','')

'"https://sun6-20.userapi.com/s/v1/ig2/m3XqTFrg5RpUsQKzmhumc9Ue4ziI9D02bciF6bjbqEJWP47xLzqfpnCvUwtzWcKelNab2mMRf3GlFVYs8LMa3NF5.jpg?size=100x100&quality=96&crop=0'

In [ ]:
https://sun9-50.userapi.com/impg/Xti8_zOWAHqRYhgRpV_Fm9wVowlBm-i5nCqU_w/VRtGoL0bU-E.jpg?size=1280x622&quality=95&sign=126afd6f299b1a75de1d0b843090a4a5&type=album

https://sun6-21.userapi.com/s/v1/ig2/wVIZNjZ9OW8Enoaz03ibkL2rlvOGaa9NucnbR8YOpA7s9BcIHcO8K6sV1nN65P3k0h


In [196]:
s = Stratosphere("sqlite:////shared/data/probe.db")
df = s.db.pandas("select * from flows where flow_id = REPLACE('78b1fa25-afa0-4013-a646-a33e73d84916', '-', '')")
ifs = InspectFlows(df)
ifs.overview()

■ Stratosphere v0.0.1 (https://stratosphere.dev) initialized
Overview: 1 flows (0.00 seconds) distributed as follows:


,host,port,method,flow_request_headers_content_type,flow_response_headers_content_type,count
0,vk.com,443,GET,NaN,text/html; charset=windows-1251,1


In [224]:
content = decode(df.iloc[0].flow_response_content)

user_ids = re.compile('"user_ids"').search(content)

re.compile('"user_ids(.*?)"').search(content).start()



content[393515:393515+50].replace("\\", "")

'"user_ids":["2460541"],"fields":["is_sber_ve'

In [201]:
user_ids = re.compile('"user_ids":(.*?),').search(content)[1]  # ['68870871']
user_id = json.loads(user_ids)[0]  # "68870871"
country = re.compile(',"country":{.*?"title":"(.*?)"}').search(content)[1]
city = re.compile(',"city":{.*?"title":"(.*?)"}').search(content)[1]
bdate = re.compile(',"bdate":"(.*?)"').search(content)[1]
skype = re.compile(',"skype":"(.*?)"').search(content)[1]


#

TypeError: 'NoneType' object is not subscriptable

In [35]:
df = s.db.pandas("select * from flows")

from bs4 import BeautifulSoup

soup = BeautifulSoup(df.iloc[0].flow_response_content, "html.parser")
soup.title.text

'Andrey Kramodanov'

In [256]:
#[l for l in content.splitlines() if "68870871" in l]

In [23]:
for idx in df[df.response_is_json].index[:1]:
    display(ifs.flow(idx).search(["Oleg"]))



[ ]     0: {
[ ]     1:   "payload": [
[ ]     2:     0,
[ ]     3:     [
[ ]     4:       [
[ ]     5:         [
[ ]     6:           456245462,
[ ]     7:           2000190017,
[ ]     8:           "https://vk.com/mp3/audio_api_unavailable.mp3?extra=agm4axn2lzhxnoitohhnnwlua1c [...]
[ ]     9:           "\u00ca\u00ee\u00f8\u00ea\u00e0",
[ ]    10:           "\u00c0\u00ed\u00ed\u00e0 \u00d1\u00e5\u00e4\u00ee\u00ea\u00ee\u00e2\u00e0",
[ ]    11:           169,
[ ]    12:           0,
[ ]    13:           0,
[ ]    14:           "",
[ ]    15:           0,
[ ]    16:           98,
[ ]    17:           "",
[ ]    18:           "[]",
[ ]    19:           "0cced7a8a6a6fc02a8//7a60a792130ca9309d///f17702532e61eebc9d/",
[ ]    20:           "https://sun6-23.userapi.com/impf/wfrgc8po4itd1tntzqjdbbcwlopouyv3sphytw/gdvked [...]
[ ]    21:           {
[ ]    22:             "duration": 169,
[ ]    23:             "content_id": "2000190017_456245462",
[ ]    24:             "puid22": 14,
[ ]   

None

In [266]:
!cat ../../users

https://vk.com/id414930480
https://vk.com/id544907148
https://vk.com/id245817023
https://vk.com/id168844559
https://vk.com/id222838908
https://vk.com/id322212960
https://vk.com/id284059860
https://vk.com/id208209071
https://vk.com/id365517529
https://vk.com/id338684201
https://vk.com/id85849014
https://vk.com/id95690402
https://vk.com/id35361174
https://vk.com/id279845015
https://vk.com/id256515381
https://vk.com/id93839112
https://vk.com/id151481861
https://vk.com/id245827059
https://vk.com/id292582377
https://vk.com/id141236356
https://vk.com/id242838000
https://vk.com/id157144910
https://vk.com/id336521927
https://vk.com/id42297895
https://vk.com/id37307897
https://vk.com/id205327224
https://vk.com/id449203823
https://vk.com/id75780708
https://vk.com/id453437492
https://vk.com/id131516120
https://vk.com/id333396799
https://vk.com/id123446984
https://vk.com/id531214740
https://vk.com/id182259723
https://vk.com/id450237193
https://vk.com/id167281499
https://vk.com/id7214049
https://vk